In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['ali.txt']


In [2]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
import string
import re

Using TensorFlow backend.


In [3]:
def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [4]:
def load_doc(filename):
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
    

In [5]:
in_filename='../input/ali.txt'

In [6]:
doc=load_doc(in_filename)

In [7]:
print(doc[0:100])

In a town of Persia lived two brothers, sons of a poor man; one named
Cassim, the other Ali Baba. Ca


In [8]:
def clean_doc(doc):
    doc = doc.replace('--', ' ')
    tokens = doc.split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [9]:
tokens=clean_doc(doc)
tokens[0:50]


['in',
 'a',
 'town',
 'of',
 'persia',
 'lived',
 'two',
 'brothers',
 'sons',
 'of',
 'a',
 'poor',
 'man',
 'one',
 'named',
 'cassim',
 'the',
 'other',
 'ali',
 'baba',
 'cassim',
 'the',
 'elder',
 'married',
 'a',
 'wife',
 'with',
 'a',
 'considerable',
 'fortune',
 'and',
 'lived',
 'at',
 'his',
 'ease',
 'but',
 'the',
 'wife',
 'of',
 'ali',
 'baba',
 'was',
 'as',
 'poor',
 'as',
 'himself',
 'they',
 'dwelt',
 'in',
 'a']

In [10]:
tk=len(tokens)
tk

957

In [11]:
length= 20 + 1
sequence=[]
for i in range(length,tk):
    seq=tokens[i-length:i]
    line=' '.join(seq)
    sequence.append(line)
    

In [12]:
out_filename = 'ali1.txt'
save_doc(sequence, out_filename)

In [13]:
len(sequence)

936

In [14]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(sequence)

In [15]:
sequence1 = tokenizer.texts_to_sequences(sequence)

In [16]:
sequence1

[[10,
  5,
  362,
  4,
  361,
  137,
  79,
  136,
  360,
  4,
  5,
  76,
  75,
  26,
  358,
  37,
  1,
  134,
  8,
  9,
  37],
 [5,
  362,
  4,
  361,
  137,
  79,
  136,
  360,
  4,
  5,
  76,
  75,
  26,
  358,
  37,
  1,
  134,
  8,
  9,
  37,
  1],
 [362,
  4,
  361,
  137,
  79,
  136,
  360,
  4,
  5,
  76,
  75,
  26,
  358,
  37,
  1,
  134,
  8,
  9,
  37,
  1,
  139],
 [4,
  361,
  137,
  79,
  136,
  360,
  4,
  5,
  76,
  75,
  26,
  358,
  37,
  1,
  134,
  8,
  9,
  37,
  1,
  139,
  140],
 [361,
  137,
  79,
  136,
  360,
  4,
  5,
  76,
  75,
  26,
  358,
  37,
  1,
  134,
  8,
  9,
  37,
  1,
  139,
  140,
  5],
 [137,
  79,
  136,
  360,
  4,
  5,
  76,
  75,
  26,
  358,
  37,
  1,
  134,
  8,
  9,
  37,
  1,
  139,
  140,
  5,
  45],
 [79,
  136,
  360,
  4,
  5,
  76,
  75,
  26,
  358,
  37,
  1,
  134,
  8,
  9,
  37,
  1,
  139,
  140,
  5,
  45,
  15],
 [136,
  360,
  4,
  5,
  76,
  75,
  26,
  358,
  37,
  1,
  134,
  8,
  9,
  37,
  1,
  139,
  140,
  5,
  4

In [17]:
vocab_size=len(tokenizer.word_index) + 1
print(vocab_size)


364


In [18]:
sequence1=np.array(sequence1)

In [19]:
sequence1

array([[ 10,   5, 362, ...,   8,   9,  37],
       [  5, 362,   4, ...,   9,  37,   1],
       [362,   4, 361, ...,  37,   1, 139],
       ...,
       [ 66,  14, 357, ...,  10, 138,   4],
       [ 14, 357,   8, ..., 138,   4,   7],
       [357,   8,   9, ...,   4,   7, 363]])

In [20]:
x,y=sequence1[:,:-1],sequence1[:,-1]

In [21]:
y = to_categorical(y, num_classes=vocab_size)

In [22]:
print("shape of training datasets(x,y)=",x.shape,y.shape)

shape of training datasets(x,y)= (936, 20) (936, 364)


In [23]:
def define_model(vocab_size, seq_length):
    model = Sequential()
    model.add(Embedding(vocab_size, 20, input_length=seq_length))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [24]:
x.shape[1]

20

In [25]:
model=define_model(vocab_size,x.shape[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 20)            7280      
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 100)           48400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 364)               36764     
Total params: 182,944
Trainable params: 182,944
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(x,y,batch_size=10,epochs=2)

Epoch 1/2
936/936 [==============================] - 9s 10ms/step - loss: 5.6214 - acc: 0.0748
Epoch 2/2
936/936 [==============================] - 7s 7ms/step - loss: 5.2764 - acc: 0.0823


In [27]:
sequence

['in a town of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim',
 'a town of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the',
 'town of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder',
 'of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married',
 'persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a',
 'lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife',
 'two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife with',
 'brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife with a',
 'sons of a poor man one named cassim the other ali baba cassim the elder married a wife with a considerable',
 'of a poor man o

In [28]:
from keras.models import load_model

In [29]:
model.save('model.h5')

In [30]:
model = load_model('model.h5')

In [31]:
sequence

['in a town of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim',
 'a town of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the',
 'town of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder',
 'of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married',
 'persia lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a',
 'lived two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife',
 'two brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife with',
 'brothers sons of a poor man one named cassim the other ali baba cassim the elder married a wife with a',
 'sons of a poor man one named cassim the other ali baba cassim the elder married a wife with a considerable',
 'of a poor man o

In [32]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
            # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)
# load cleaned text sequences
in_filename = 'ali1.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
# select a seed text
seed_text = 'in a town of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim'
print(seed_text + '\n')
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 60)
print(generated)


in a town of persia lived two brothers sons of a poor man one named cassim the other ali baba cassim

the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
